**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from class_utils import error_histogram
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/p5q7gzupa2ndw55/sigmoid_regression_data.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Regresia založená na neurónových sieťach

V tomto notebooku ukážeme, ako sa jednoduchá neurónová sieť vytvorená pomocou balíčka `PyTorch` dá aplikovať na regresné úlohy. Vytvoríme si veľmi jednoduchý viacvrstvý perceptrón, natrénujeme ho a vizualizujeme si výsledky.

### Dátová množina

Začnime definovaním regresnej úlohy. Načítame si dátovú množinu z CSV súboru – dáta sú zašumené vzorky zo sigmoidnej (logistickej) krivky. Keďže sme sa s podobnými typmi dát už v predchádzajúcich notebookoch stretli, nebudeme sa v tomto prípade podrobne venovať načítaniu a predspracovaniu dát a kód nasledujúcej bunky ponecháme skrytý.



In [ ]:
#@title -- Data Loading and Preprocessing; X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

# we create a discretized version of the y column
# to allow for stratification
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

# we split the dataset into train and test
df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

# we specify the inputs and the outputs
categorical_inputs = []
numeric_inputs = ['x']
output = ['y']

# we create the pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='constant', fill_value='MISSING'),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# we fit and apply the pipeline on the train set
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values

# we apply the same pipeline to the test set,
# taking care to use transform and not fit_transform
X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values

# we plot the data for visual inspection
plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_data.pdf", bbox_inches='tight', pad_inches=0)

Okrem štandardného predspracovania výsledky ešte transformujeme do dátových typov, ktoré očakáva PyTorch: t.j. na PyTorch tenzory (podobné `numpy` poliam, ale s podporou pre autodiff) 32-bitových float-ov.



In [ ]:
X_train = torch.as_tensor(X_train, dtype=torch.float32)
Y_train = torch.as_tensor(Y_train, dtype=torch.float32)
X_test = torch.as_tensor(X_test, dtype=torch.float32)
Y_test = torch.as_tensor(Y_test, dtype=torch.float32)

### Výber zariadenia a prenos našich údajov

Naša neurónová sieť môže bežať na niekoľkých rôznych druhoch zariadení. V predvolenom nastavení všetko beží na procesore (CPU), ale PyTorch podporuje aj určité druhy grafických kariet (GPU), ktorých použitie môže veľmi výrazne urýchliť výpočty. Existujú aj iné špeciálne zariadenia, ako sú TPU, FPGA atď., ale ak chcete model spustiť na nich, budete typicky potrebovať nejaké rozšírenia PyTorch-u.

Teraz teda špecifikujme, aký druh zariadenia chceme použiť: povedzme, že chceme použiť GPU, ak je k dispozícii, a CPU, ak nie je. Dostupnosť GPU môžeme skontrolovať pomocou `torch.cuda.is_available`. Za zmienku stojí, že na počítačoch s viacerými GPU si môžete vybrať, ktoré konkrétne GPU (alebo skupinu GPU) chcete použiť – to je však už nad rámec tohto notebooku.

Tu jednoducho vyberieme `"cuda"` (GPU, takto pomenované podľa framework-u CUDA od Nvidie), ak platí `torch.cuda.is_available()` a `"cpu"` inak .



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Keď na spustenie modelu používame určité zariadenie, musíme sa uistiť, že aj svoje dáta prenesieme do pamäte tohto zariadenia. To sa dá ľahko urobiť pomocou metódy `.to(device)` poskytovanej PyTorch tenzormi. Ak chceme dáta teraz preniesť do zariadenia, ktoré sme zvolili, môžeme spustiť:



In [ ]:
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

### Vytvorenie neurónovej siete a trénovanie

Ak chceme vytvoriť našu neurónovú sieť, budeme dediť zo základnej triedy `nn.Module`. Všetky vrstvy s trénovateľnými parametrami vytvárame v konštruktore a priraďujeme ich ako atribúty našej sieti. Spôsob, akým sú vrstvy jedna s druhou prepojené a akým vypočítavajú zo svojich vstupov výstup, je definovaný v metóde `forward`. Neurónová sieť musí mať určitý pevný počet vstupných a výstupných neurónov. Počet vstupov bude samozrejme rovný počtu stĺpcov v našej `X_train` množine, zatiaľ čo počet výstupov bude rovný počtu stĺpcov vo `Y_train` množine.

Pripomeňme, že v neurónových sieťach určených na regresiu sa typicky **necháva posledná vrstva lineárna**  (bez aktivačnej funkcie), aby bola schopná produkovať neohraničené výstupy a nemusela sa učiť invertovať vplyv nelineárnych aktivačných funkcií keď ich tvar nie je vhodný pre regresnú úlohu.



In [ ]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.fc1 = nn.Linear(num_inputs, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, num_outputs)

    def forward(self, x):
        y = self.fc1(x)
        y = torch.relu(y)
        
        y = self.fc2(y)
        y = torch.relu(y)
        
        y = self.fc3(y)        
        return y

Teraz sme pripravení vytvoriť model. Všimnite si, že model je opäť potrebné preniesť do zvoleného zariadenia, čo sa robí presne rovnakým spôsobom, aký sme použili s údajmi: volaním `.to(device)`.



In [ ]:
num_inputs = X_train.shape[1]
num_outputs = Y_train.shape[1]

model = Net(num_inputs, num_outputs)
model = model.to(device)

#### Spustenie siete

Ak sme urobili všetko správne, teraz by sme mali byť schopní spustiť model na na našich dátach. Skúsme to s prvými 5 riadkami z `X_train`.



In [ ]:
y = model(X_train[:5, ...])
y

#### Tenzory, gradienty, detaching

Možno ste si všimli `grad_fn` vo výpise nášho tenzora. Ako už bolo spomenuté, PyTorch tenzory majú vstavanú podporu pre autodiff. Keď na nich spustíte nejaké operácie, priebežne sa vytvára výpočtový graf, ktorým je potom možné robiť spätné šírenie.

Ak sa chystáte s vašimi tenzormi vykonávať ďalšie operácie, ktoré nie sú súčasťou tréningového procesu, ako je logovanie hodnôt chybovej funkcie, zobrazovanie grafov atď., je dobré extrahovať len údaje a zbaviť sa výpočtového grafu skôr než urobíte čokoľvek iné. Dá sa to realizovať pomocou `.detach()`; keď si zobrazíme tenzor, uvidíme, že časť `grad_fn` zmizla.



In [ ]:
y.detach()

#### Konverzia na NumPy

Ak chcete tenzor konvertovať na `numpy` pole, môžete na ňom spustiť metodu `.numpy()`. Keďže tenzor môže mať pripojené informácie o gradiente, vo všeobecnosti je dobré najskôr zavolať `.detach`. Okrem toho môže byť tenzor na inom zariadení, takže pre istotu zvyčajne budete najprv voláme `.cpu()`, aby sme ho preniesli späť do CPU.

T.j. toto je spoľahlivý spôsob konverzie PyTorch tenzorov na numpy polia:



In [ ]:
y_np = y.detach().cpu().numpy()
y_np

Podobne, ak váš tenzor obsahuje skalár, môžete ho extrahovať jednoducho volaním `.item()`:



In [ ]:
y_scalar = y.mean()
s = y_scalar.detach().cpu().item()
s

#### Spustenie bez gradientov

Keď spúšťate model mimo tréningu, zvyčajne nebudete potrebovať podporu autodiffu a výpočtový graf. V takýchto prípadoch je dobrý nápad výpočtový graf vypnúť, pretože jeho vytvorenie tiež kladie určité výpočtové nároky. Realizovať to možno tak, že PyTorch volania umiestnite do kontextu `torch.no_grad()`, napr.:



In [ ]:
with torch.no_grad():
    y = model(X_train[:5, ...])

y

Všimnite si, že tenzor teraz nemá `grad_fn`, hoci keď sme na ňom nespustili `.detach()`. Dôvodom je, že kontext `torch.no_grad()` zabránil tomu, aby sa výpočtový graf vôbec zostavil.

#### Režim train vs. režim eval

PyTorch má viacero špeciálnych vrstiev, ktoré sa počas tréningu správajú inak než počas inferencie. Existuje napríklad vrstva dropout, ktorá počas tréningu náhodne vypína určitú časť svojich neurónov, aby tým pomohla predísť preučeniu sa siete. Počas inferencie je toto správanie, samozrejme, deaktivované, pretože nechceme negatívne ovplyvniť kvalitu predikcií.

Na podporu oboch týchto prípadov disponujú PyTorch modely dvoma odlišnými režimami:

* **Tréningový režim:**  Pri trénovaní prepnete model do tréningového režimu volaním `model.train()`;
* **Režim hodnotenia:**  Keď spúšťate inferenciu, prepnete ho do režimu hodnotenia volaním `model.eval()`.


In [ ]:
# during training:
model.train()
y = model(X_train[:5, ...])

# during inference:
model.eval()
with torch.no_grad():
    y = model(X_test[:5, ...])

### Tréningová slučka

Napokon už ostáva len spustiť učenie. V PyTorch-i je na to potrebné pomerne veľké množstvo kódu: musíme vytvoriť chybovú funkciu, optimalizátor a predovšetkým od nuly napísať celú tréningovú slučku. Tento prístup však poskytuje veľkú flexibilitu, čo bude veľmi užitočné pri vytváraní a trénovaní zložitejších modelov.

V neskorších príkladoch si ukážeme, ako trénovať na minidávkach a budeme môcť tréningovú slučku vylepšiť ďalšími sofistikovanejšími funkciami, ako sú scheduling rýchlosti učenia (learning rate scheduling), skoré ukončenie učenia (early stopping), načítanie dát za behu a zväčšovanie dátovej množiny (data augmentation), atď. Tu však začneme veľmi jednoduchou verziou. Keďže sú naše dáta maličké, tréning budeme realizovať v plne dávkovom režime, t.j. všetky tréningové dáta budeme vkladať do modelu naraz.

#### Vytvorenie optimalizátora

Ako optimalizátor použijeme `Adam`. Pri jeho konštrukcii musíme špecifikovať:

* aké parametre sa budú optimalizovať – zadáme `model.parameters()`, teda parametre nášho modelu;
* aká bude rýchlosť učenia.
#### Konštrukcia chybovej funkcie

Ako chybovú funkciu budeme používať **strednú kvadratickú chybu** , ktorá je bežnou voľbou pri regresných úlohách. Skonštruujeme ju jednoducho pomocou `nn.MSELoss` balíčka PyTorch.

Zvyšnú časť kódu tréningovej slučky vysvetlíme prostredníctvom komentárov v nasledujúcej bunke.



In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_train = [] # tu si uložíme tréningové chyby na neskoršie vykreslenie

# budeme trénovať zopár epoch
for epoch in range(1000):
    # model dáme do tréningového režimu
    model.train()

    # spustíme model na dátach
    y = model(X_train)

    # zmeriame hodnotu chybovej funkcie a uložíme si ju
    loss = criterion(y, Y_train)
    loss_train.append(loss.detach().item())

    # vymažeme všetky gradienty, ktoré sa vypočítali
    # v rámci predošlej iterácie
    optimizer.zero_grad()

    # realizujeme spätné šírenie chyby
    loss.backward()

    # pomocou optimalizátora aktualizujeme váhy
    optimizer.step()

    # raz za čas zobrazíme report o progrese
    if epoch % 100 == 0:
        print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:])}")

print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:])}")

### Testovanie

Keď sme model natrénovali, sme pripravení otestovať, ako dobre funguje. Model nezabudneme najprv uviesť do evaluačného režimu pomocou `model.eval()` a spustiť ho v rámci `torch.no_grad()`, aby sa zbytočne nevytváral výpočtový graf.

Na vyhodnotenie si vypočítame MSE, MAE a zobrazíme náš zvyčajný histogram chýb so štandardizovanou škálou.

#### Na tréningových dátach



In [ ]:
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_train_cpu = model(X_train).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_train_cpu, y_train_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_train_cpu, y_train_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_train_cpu, y_train_cpu, Y_fit_scaling=Y_train_cpu)

#### Na testovacích dátach



In [ ]:
Y_test_cpu = Y_test.cpu()

model.eval()
with torch.no_grad():
    y_test_cpu = model(X_test).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test_cpu, y_test_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test_cpu, y_test_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_test_cpu, y_test_cpu, Y_fit_scaling=Y_train)

Tieto výsledky indikujú, že model funguje celkom dobre – chyby sú nízke na tréningovej aj testovacej množine. Keďže pracujeme s 2D dátami, vykreslime si body aj v pôvodnom priestore.

Stále môžeme pozorovať drobné artefakty v niektorých častiach krivky, ale celkový tvar by mal byť zachytený vcelku dobre, ak sú naše výsledky na tréningovej aj testovacej množine dobré.



In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = min(torch.min(X_train), torch.min(X_test))
x_max = max(torch.max(X_train), torch.max(X_test))
xx = torch.linspace(x_min, x_max, 250).reshape((-1, 1))

model.eval()
with torch.no_grad():
    yy = model(xx.to(device))
    yy = yy.cpu()

plt.scatter(X_train.cpu(), Y_train.cpu(), marker='x', label="training data")
plt.scatter(X_test.cpu(), Y_test.cpu(), c='r', label="testing data")

plt.plot(xx, yy, label="regression curve", c='k')

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

plt.savefig("output/regression.pdf", bbox_inches="tight", pad_inches=0)